<a href="https://colab.research.google.com/github/Horax02/Artificial_Vision-Mitnicks/blob/main/EfficientNet_Fallido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB7
# Load the Pascal VOC dataset using TensorFlow Datasets (TFDS)
dataset, info = tfds.load('voc', split='train+validation', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating test examples...:   0%|          | 0/4952 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/voc/2007/4.0.0.incomplete3RW7BQ/voc-test.tfrecord*...:   0%|          | 0/…

Generating train examples...:   0%|          | 0/2501 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/voc/2007/4.0.0.incomplete3RW7BQ/voc-train.tfrecord*...:   0%|          | 0…

Generating validation examples...:   0%|          | 0/2510 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/voc/2007/4.0.0.incomplete3RW7BQ/voc-validation.tfrecord*...:   0%|        …

Dataset voc downloaded and prepared to /root/tensorflow_datasets/voc/2007/4.0.0. Subsequent calls will reuse this data.


In [ ]:
from tensorflow.keras.applications import EfficientNetB7
# Define the number of classes in the Pascal VOC dataset
num_classes = info.features['objects']['label'].num_classes

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
# Preprocess and augment the dataset as needed

# Set up the EfficientNetB7 pre-trained model
base_model = EfficientNetB7(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Add custom layers for object detection
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

258076736/258076736 [==============================] - 4s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the Pascal VOC dataset
# Make sure to specify the appropriate data generator and training parameters

# Save the trained model weights
model.save_weights('trained_model_weights.h5')

# Save the model architecture to JSON
model_config = model.get_config()
with open('trained_model_architecture.json', 'w') as json_file:
    json_file.write(str(model_config))

In [ ]:
import numpy as np
import tensorflow as tf
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util
import cv2

# Path to the frozen inference graph
PATH_TO_FROZEN_GRAPH = 'path/to/your/frozen_inference_graph.pb'

# Path to the label map
PATH_TO_LABEL_MAP = 'path/to/your/label_map.pbtxt'

# Load the frozen inference graph
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Load the label map
label_map = label_map_util.load_labelmap(PATH_TO_LABEL_MAP)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def detect_objects(image_path):
    # Load the image
    image = cv2.imread(image_path)
    image_expanded = np.expand_dims(image, axis=0)

    # Get handles to input and output tensors
    with detection_graph.as_default():
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            # Run inference
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_expanded})

            # Visualize the detections
            vis_util.visualize_boxes_and_labels_on_image_array(
                image,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=2,
                min_score_thresh=0.5)

            # Display the image with detections
            cv2.imshow('Object Detection', image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

# Example usage
image_path = 'path/to/your/image.jpg'
detect_objects(image_path)


1/1 [==============================] - 1s 540ms/step


TypeError: ignored